# Наша задача:

## На основании информации из датасета df оценить прибыльность рейсов из Анапы в зимнее время и решить от каких рейсов можно отказаться.

In [1]:
#Импортируем нужные нам библиотеки
import pandas as pd
!pip install geopy
from geopy.distance import geodesic

In [2]:
# Прочитаем файл csv с результатами sql запроса в переменную df
df = pd.read_csv('C:\My_Job\Data Scientist_Skill Factory\Folders with data/Unit_4_Project/query_result_2021-08-02T21_26_54.042483Z.csv')

# НАШ ОРИГИНАЛЬНЫЙ ДАТАСЕТ С РЕЗУЛЬТАТАМИ SQL ЗАПРОСА.
# ЭТО НЕ ФИНАЛЬНАЯ ВЕРСИЯ:

In [3]:
df # Запускаем блок если хотим видеть датасет на этом этапе.

,ticket_no,flight_id,flight_no,scheduled_departure,duration,status,seat_no,boarding_no,amount,fare_conditions,model,departure_airport,arrival_airport,airport_name,city,longitude,latitude
0,5432320828,136612,PG0480,2017-02-21 09:25:00+00,00:50:00,Arrived,6C,6,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
1,5432374656,136609,PG0480,2017-01-08 09:25:00+00,00:50:00,Arrived,6C,54,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
2,5432848786,136800,PG0480,2016-12-09 09:25:00+00,00:51:00,Arrived,6C,55,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
3,5432376664,136844,PG0480,2017-02-28 09:25:00+00,00:50:00,Arrived,6C,78,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
4,5432375495,136869,PG0480,2017-01-31 09:25:00+00,00:50:00,Arrived,6C,88,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18346,5432375779,136600,PG0480,2017-02-08 09:25:00+00,00:50:00,Arrived,6C,18,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
18347,5432320217,136666,PG0480,2017-02-09 09:25:00+00,00:50:00,Arrived,6C,4,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
18348,5432320465,136605,PG0480,2017-02-15 09:25:00+00,00:51:00,Arrived,6C,95,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
18349,5432320534,136827,PG0480,2017-02-18 09:25:00+00,00:51:00,Arrived,6C,24,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799


In [4]:
df_original = df.copy()

In [5]:
# Нам известны географические координаты аэропорта вылета - Анапы (37.35 с.ш. 45 в.д.) и всех аэропортов прибытия. 
#Таким образом мы можем рассчитать для каждого рейса расстояние между аэропортом вылета и аэропортом прибытия.
df['distance'] = 0.0 # Создаем новый столбец с расстояниями от Анапы до аэропортов прилета и заполняем его нулями
for i in range (0, len(df)): # Рассчитываем расстояния для каждого рейса
    df.distance[i] = geodesic((37.35, 45.0),(df.longitude[i], df.latitude[i])).km
del df['longitude']
del df['latitude']# Убираем географические координаты, они были нужны только для рассчета расстояний


df.duration = df.duration.apply(lambda x: round((int(x[0:2])+int(x[3:5])/60), 2))# Перевели данные в столбце duration (про
#должительность полета) в удобный нам вид для перемножения с часовым расходом топлива)

# В интернете я нашел информацию о часовом расходе топлива. Для Sukhoi Superjet 100 это 1615 кг/час, а для Boeing 737-300
# это 2600 кг/час. Из интернета узнал цену топлива - 31000 рублей за тонну (31 рубль за килограмм) авиационного керосина 
# марки ТС-1. Условимся считать, что заправляли этой маркой по этой цене. Зная фактическую продолжительность полета мы можем
# посчитать на какую сумму сжигалось топлива в каждом полете.

df['fired_fuel_price'] = 0 # Создали заполненный нулями столбец fired_fuel_price куда запишем цену сожженного в полете топлива
for i in range (0,len(df)): # Считаем цену сожженного в полете топлива и пишем ее в столбец fired_fuel_price
    if df.model[i] == 'Boeing 737-300':
        df.fired_fuel_price[i] = df.duration[i]*2600*31
    else:
        df.fired_fuel_price[i] = df.duration[i]*1615*31
    

<ipython-input-5-ff75e56ea46f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.distance[i] = geodesic((37.35, 45.0),(df.longitude[i], df.latitude[i])).km
<ipython-input-5-ff75e56ea46f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fired_fuel_price[i] = df.duration[i]*1615*31
<ipython-input-5-ff75e56ea46f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fired_fuel_price[i] = df.duration[i]*2600*31


In [6]:
df_backup = df.copy()

In [7]:
#Рассчитаем суммарную стоимость проданных билетов на каждый рейс
del df['ticket_no']
del df['flight_no']
del df['scheduled_departure']
del df['duration']
del df['status']
del df['fare_conditions']
del df['model']
del df['departure_airport']
del df['arrival_airport']
del df['airport_name']
del df['city']
del df['distance']
del df['fired_fuel_price']
del df['seat_no']
del df['boarding_no']
s = df.groupby(['flight_id']).sum().amount
s# Получили серию s, где для каждого рейса (flight_id) рассчитана суммарная стоимость всех проданных на этот рейс билетов.

flight_id
136119    1653000.0
136120    1605400.0
136122    1431000.0
136130    1556600.0
136131    1812800.0
            ...    
136943     733800.0
136951     720600.0
136953     765300.0
136956     746400.0
136961     759000.0
Name: amount, Length: 180, dtype: float64

In [8]:
df = df_backup

In [9]:
# теперь добавим в наш датасет столбец с суммарной стоимостью всех проданных на рейс билетов. Считается долго!!!
df['total_ticket_price_for_flight'] = 0.0# Создаем столбец total_ticket_price_for_flight заполненный нулями
for i in range (0, len(s)):
    for j in range (0, len(df)):
        if s.index[i] == df.flight_id[j]:
            df.total_ticket_price_for_flight[j] = s[s.index[i]]
        else: df.flight_id[j] = df.flight_id[j]

<ipython-input-9-cd0ecc1be8a6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: df.flight_id[j] = df.flight_id[j]
<ipython-input-9-cd0ecc1be8a6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.total_ticket_price_for_flight[j] = s[s.index[i]]


In [10]:
# Теперь, зная для каждого рейса стоимость сожженного топлива и суммарную стоимость проданных билетов мы можем рассчитать
# прибыль компании с этого рейса.
df['profit_from_flight'] = 0.0 # Создали столбец profit_from_flight заполненный нулями.
df.profit_from_flight = df.total_ticket_price_for_flight - df.fired_fuel_price

In [ ]:
df # Запускаем блок если хотим видеть датасет на этом этапе.

In [11]:
df_full = df.copy()

In [12]:
# Проверяем по каким маршрутам шли рейсы самолетов Boeing 737-300. 
df[df.model == 'Boeing 737-300'].flight_no.value_counts()

PG0252    10210
Name: flight_no, dtype: int64

In [13]:
# Проверяем по каким маршрутам шли рейсы самолетов Sukhoi Superjet-100. 
df[df.model == 'Sukhoi Superjet-100'].flight_no.value_counts()

PG0480    8141
Name: flight_no, dtype: int64

In [14]:
# Видно, что все рейсы маршрута Анапа-Москва выполнены самолетами модели Boeing 737-300, а все рейсы маршрута 
#Анапа-Белгород самолетами Sukhoi Superjet-100. Таким образом нам достаточно посчитать среднее значение прибыли с одного 
#рейса по каждому направлению (если бы один и тот же маршрут выполнялся разными моделями самолетов, то был бы смысл 
#группировать рейсы по моделям самолетов и уже в этих группах с уникальной комбинацией маршрута и модели самолета 
#считать среднее).
# Удаляем лишние столбцы
del df['ticket_no']
del df['flight_id']
del df['scheduled_departure']
del df['duration']
del df['status']
del df['seat_no']
del df['boarding_no']
del df['amount']
del df['fare_conditions']
del df['departure_airport']
del df['airport_name']
del df['distance']
del df['fired_fuel_price']
del df['total_ticket_price_for_flight']

In [15]:
#Посчитаем среднюю прибыль с одного рейса PG0252 Анапа-Москва
s = df.groupby(['flight_no']).mean().profit_from_flight 
display(s.index[0])
display(round(s[0], 2))

'PG0252'

1524773.45

In [16]:
#Посчитаем среднюю прибыль с одного рейса PG0480 Анапа-Белгород
s = df.groupby(['flight_no']).mean().profit_from_flight 
display(s.index[1])
display(round(s[1], 2))

'PG0480'

675337.57

Средняя прибыль с одного рейса для маршрута PG0252 Анапа-Москва (Шереметьево), составляет 1524773 рубля 45 копеек, а для маршрута PG0480 Анапа-Белгород 675337 рублей 57 копеек. Мы не будем отказываться ни от одного из этих маршрутов. Но считаю нужным напомнить руководству компании о тринадцати рейсах маршрута PG0194 Анапа - Новокузнецк, по которым у нас нет данных и которые, если выяснится, что они летают порожняком необходимо убрать.

In [17]:
df_full# Выведем наш итоговый датасет, полученный после дополнения оригинального датасета всеми данными и 
#выполнения рассчета всех параметров.

,ticket_no,flight_id,flight_no,scheduled_departure,duration,status,seat_no,boarding_no,amount,fare_conditions,model,departure_airport,arrival_airport,airport_name,city,distance,fired_fuel_price,total_ticket_price_for_flight,profit_from_flight
0,5432320828,136612,PG0480,2017-02-21 09:25:00+00,0.83,Arrived,6C,6,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,746400.0,704847.0
1,5432374656,136609,PG0480,2017-01-08 09:25:00+00,0.83,Arrived,6C,54,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,639300.0,597747.0
2,5432848786,136800,PG0480,2016-12-09 09:25:00+00,0.85,Arrived,6C,55,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,42555,645000.0,602445.0
3,5432376664,136844,PG0480,2017-02-28 09:25:00+00,0.83,Arrived,6C,78,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,575100.0,533547.0
4,5432375495,136869,PG0480,2017-01-31 09:25:00+00,0.83,Arrived,6C,88,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,765300.0,723747.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18346,5432375779,136600,PG0480,2017-02-08 09:25:00+00,0.83,Arrived,6C,18,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,765300.0,723747.0
18347,5432320217,136666,PG0480,2017-02-09 09:25:00+00,0.83,Arrived,6C,4,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,664500.0,622947.0
18348,5432320465,136605,PG0480,2017-02-15 09:25:00+00,0.85,Arrived,6C,95,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,42555,765300.0,722745.0
18349,5432320534,136827,PG0480,2017-02-18 09:25:00+00,0.85,Arrived,6C,24,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,42555,765300.0,722745.0
